In [ ]:
!pip install lac
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Created wheel for lac: filename=LAC-2.1.2-py2.py3-none-any.whl size=64814699 sha256=7aae46d2a7f761a9bf99e6e16cac90e76d96f8a59d7c63eba467dfd1b3d9eb28
  Stored in directory: /root/.cache/pip/wheels/5b/a0/02/8fac032f2babbef354a8911bb7ad903f9cfc849458fdeab295
Successfully built lac
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# # 2. Use gdown tool to download from Google Drive link
!pip install gdown
import gdown
url = 'https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download'
output = '/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'
gdown.download(url, output, quiet=False)

Mounted at /content/drive


Downloading...
From: https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download
To: /content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx
100%|██████████| 557M/557M [00:08<00:00, 65.1MB/s]


'/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# loading Data
news_data = pd.read_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news_data.head()

,NewsID,Title,NewsContent,NewsSource
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,中国证券报
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,人民日报
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,杭州新希望
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,源达投资
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,全景网


In [ ]:
news_data.shape

(1037035, 4)

In [ ]:
import re
import pandas as pd
import json
from rapidfuzz import process, fuzz

# Preliminary
json_path = '/content/drive/MyDrive/DSAA5002PROJECT/dsaa5002project/dsaa5002_project/A_share_list.json'
news_data['Title'] = news_data['Title'].astype(str)
news_data['NewsContent'] = news_data['NewsContent'].astype(str)
df = news_data[640000:840000]

# Read json file
with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)

abbreviation_to_name = {
    'ST亚星': '亚星化学',
    'SST丰华': '丰华股份',
    '中信': '中信证券',
    '中金': '中金公司',
    '中石油': '中国石油',
    '中石化': '中国石化',
    '中钢': '中钢国际',
    '招行': '招商银行',
    '工行': '工商银行',
    '交行': '交通银行',
    '中行': '中国银行',
    '建行': '建设银行',
    '国寿': '中国人寿',
    '平保': '中国太保',
    '南航': '南方航空',
    '海航': '海南航空',
    '万科': '万科A',
    '茅台': '贵州茅台',
    '海尔': '青岛海尔'
}

# Prepare mapping of stock names, full names and codes
stock_info = {}
for stock in stock_list:
    name = stock['name']
    fullname = re.sub('股份有限公司$', '', stock['fullname'])
    code = re.sub('\D', '', stock['code'])
    stock_info[name] = name
    stock_info[fullname] = name
    stock_info[code] = name

# Prepare mapping of stock names, full names and codes
for abbreviation, name in abbreviation_to_name.items():
    stock_info[abbreviation] = name

from LAC import LAC

lac = LAC(mode='lac')

def extract_org_names(text):
    lac_result = lac.run(text)
    words = lac_result[0]
    tags = lac_result[1]

    org_entities = [word for word, tag in zip(words, tags) if tag == 'ORG']
    unique_org_entities = list(set(org_entities))
    return unique_org_entities

def find_stock_names(org_names, stock_names, threshold=90):
    matched_names = set()
    for org_name in org_names:
        if org_name in ['中国人民银行', '中国央行', '央行', '中金所', '中国金融期货交易所', '公司', '京东', '美团']:
            continue
        else:
            best_match = process.extractOne(org_name, stock_names, scorer=fuzz.QRatio, score_cutoff=threshold)
            if best_match:
                if set(best_match[0]).issubset(set(org_name)) or set(org_name).issubset(set(best_match[0])):
                    matched_names.add(best_match[0])
    return matched_names

with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)
stock_names = [stock['name'] for stock in stock_list]

def extract_stock_names(text, stock_info, stock_names):
    if not isinstance(text, str):
        return ''

    org_names = extract_org_names(text)

    fuzzy_matched_names = find_stock_names(org_names, stock_names)

    text_no_stars = text.replace('*', '')

    direct_matched_names = set()
    for variant, name in stock_info.items():
        if variant in text_no_stars:
            direct_matched_names.add(name)

    matched_names = direct_matched_names.union(fuzzy_matched_names)

    return ','.join(matched_names)

df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')

def remove_duplicate_names(names):
    unique_names = set(names.split(','))
    return ','.join(unique_names)

# Apply the function to the 'Explicit_Company' column to remove duplicates
df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)

# Now you can proceed with filtering the DataFrame
filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) > 0)]

# Output the result
print(len(filtered_df))


<ipython-input-10-42e42b93eb90>:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
<ipython-input-10-42e42b93eb90>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')  # 移除多余的逗号


110727


<ipython-input-10-42e42b93eb90>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)


In [ ]:
filtered_df

,NewsID,Title,NewsContent,NewsSource,Explicit_Company
640000,640001,中京电子：公司与HUAWEI汽车尚无合作,每经AI快讯，有投资者在投资者互动平台提问：华为问界新的标识正式发布上线，请问贵司对华为汽车...,每日经济新闻,中京电子
640001,640002,大中矿业：公司已安排业务部门开始办理矿权的延期相关工作,每经AI快讯，有投资者在投资者互动平台提问：林总，贵公司大坝沟探矿权即将到期，请问相关的探转...,每日经济新闻,大中矿业
640002,640003,天风证券给予家联科技增持评级 业绩符合预期 订单有望逐步回暖 产能释放奠定规模扩张,天风证券03月24日发布研报称，给予家联科技(301193.SZ，最新价：34.62元)增持...,每日经济新闻,天风证券
640003,640004,东方证券给予智飞生物买入评级 智飞生物年报点评 近期获10份券商研报关注 目标均价涨幅34.3%,东方证券03月23日发布研报称，给予智飞生物(300122.SZ，最新价：86.67元)买入...,每日经济新闻,"东方证券,智飞生物"
640004,640005,德邦证券给予沪电股份买入评级 Q4业绩高成长 有望受益AI算力需求爆发,德邦证券03月24日发布研报称，给予沪电股份(002463.SZ，最新价：21.1元)买入评...,每日经济新闻,沪电股份
...,...,...,...,...,...
839988,839989,机构今日买入这17股 抛售中航电测5.18亿元丨龙虎榜,盘后数据显示，7月27日龙虎榜中，共29只个股出现了机构的身影，有17只股票呈现机构净买入，...,第一财经,"山水比德,中南建设,迪马股份,中航电测"
839989,839990,北向资金今日净买入宁德时代15.18亿元,北向资金今日净买入宁德时代15.18亿元、贵州茅台7.80亿元，净卖出隆基绿能6.65亿元、...,证券时报,"宁德时代,贵州茅台,长安汽车"
839992,839993,2022年全球百大集装箱港口发布 榜单前十名中有7个来自中国,国际知名航运媒体《劳氏日报》近日公布2022年全球百大集装箱港口排名，27个中国港口上榜，当...,澎湃新闻,"连云港,南京港,广州港,宁波港,青岛港,天津港"
839994,839995,诗画浙江两岸文创设计共创营开启 浙台设计师共促文创发展,中新网杭州7月27日电(王题题)7月27日，以“两岸共创，浙里出发”为主题的2023诗画浙江...,中国新闻网,起步股份


In [ ]:
Be_filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) == 0)]
Be_filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Be_filtered1_part4.xlsx' ,encoding='utf-8')
filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Task1_part1-4.xlsx' ,encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
